### Importing all libraries that will be required to the project

In [115]:
import numpy as np
import geocoder
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### Getting content table of neighbourhood area of Toronto from the wikipidia page and scrapping the table using BeautifulSoup

In [116]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df=df[0].to_json(orient='records')
#print( tabulate(df[0], headers='keys', tablefmt='psql'))

### Checking the Dataframe

In [117]:
df

'[{"0":"Postcode","1":"Borough","2":"Neighbourhood"},{"0":"M1A","1":"Not assigned","2":"Not assigned"},{"0":"M2A","1":"Not assigned","2":"Not assigned"},{"0":"M3A","1":"North York","2":"Parkwoods"},{"0":"M4A","1":"North York","2":"Victoria Village"},{"0":"M5A","1":"Downtown Toronto","2":"Harbourfront"},{"0":"M5A","1":"Downtown Toronto","2":"Regent Park"},{"0":"M6A","1":"North York","2":"Lawrence Heights"},{"0":"M6A","1":"North York","2":"Lawrence Manor"},{"0":"M7A","1":"Queen\'s Park","2":"Not assigned"},{"0":"M8A","1":"Not assigned","2":"Not assigned"},{"0":"M9A","1":"Etobicoke","2":"Islington Avenue"},{"0":"M1B","1":"Scarborough","2":"Rouge"},{"0":"M1B","1":"Scarborough","2":"Malvern"},{"0":"M2B","1":"Not assigned","2":"Not assigned"},{"0":"M3B","1":"North York","2":"Don Mills North"},{"0":"M4B","1":"East York","2":"Woodbine Gardens"},{"0":"M4B","1":"East York","2":"Parkview Hill"},{"0":"M5B","1":"Downtown Toronto","2":"Ryerson"},{"0":"M5B","1":"Downtown Toronto","2":"Garden District

### Loading the dataframe as Json file

In [118]:
toronto_data=json.loads(df)

In [119]:
toronto_data

[{'0': 'Postcode', '1': 'Borough', '2': 'Neighbourhood'},
 {'0': 'M1A', '1': 'Not assigned', '2': 'Not assigned'},
 {'0': 'M2A', '1': 'Not assigned', '2': 'Not assigned'},
 {'0': 'M3A', '1': 'North York', '2': 'Parkwoods'},
 {'0': 'M4A', '1': 'North York', '2': 'Victoria Village'},
 {'0': 'M5A', '1': 'Downtown Toronto', '2': 'Harbourfront'},
 {'0': 'M5A', '1': 'Downtown Toronto', '2': 'Regent Park'},
 {'0': 'M6A', '1': 'North York', '2': 'Lawrence Heights'},
 {'0': 'M6A', '1': 'North York', '2': 'Lawrence Manor'},
 {'0': 'M7A', '1': "Queen's Park", '2': 'Not assigned'},
 {'0': 'M8A', '1': 'Not assigned', '2': 'Not assigned'},
 {'0': 'M9A', '1': 'Etobicoke', '2': 'Islington Avenue'},
 {'0': 'M1B', '1': 'Scarborough', '2': 'Rouge'},
 {'0': 'M1B', '1': 'Scarborough', '2': 'Malvern'},
 {'0': 'M2B', '1': 'Not assigned', '2': 'Not assigned'},
 {'0': 'M3B', '1': 'North York', '2': 'Don Mills North'},
 {'0': 'M4B', '1': 'East York', '2': 'Woodbine Gardens'},
 {'0': 'M4B', '1': 'East York', '2'

### Converting the df into Pandas Dataframe

In [120]:
toronto_neighborhood=pd.DataFrame(toronto_data )

In [121]:
toronto_neighborhood.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [122]:
toronto_neighborhood.shape

(289, 3)

### Naming the columns as Postcode, Borough and Neighbourhood

In [123]:
toronto_neighborhood.columns=['Postcode', 'Borough',"Neighborhood"]

### Droping the first row as it has name of columns in it which we already named

In [124]:
toronto_neighborhood.drop(0,axis=0,inplace=True)

### resetting the index

In [125]:
toronto_neighborhood.reset_index(inplace=True)

In [126]:
toronto_neighborhood.head()

,index,Postcode,Borough,Neighborhood
0,1,M1A,Not assigned,Not assigned
1,2,M2A,Not assigned,Not assigned
2,3,M3A,North York,Parkwoods
3,4,M4A,North York,Victoria Village
4,5,M5A,Downtown Toronto,Harbourfront


### Dropping the index column which we don't need

In [127]:
toronto_neighborhood.drop(['index'], axis=1, inplace=True)

In [128]:
toronto_neighborhood.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Selecting rows only where Borough is assigned

In [129]:
toronto_neighborhood1=toronto_neighborhood[toronto_neighborhood["Borough"]!="Not assigned"]

In [130]:
toronto_neighborhood1.reset_index(inplace=True)

In [131]:
toronto_neighborhood1.drop("index", axis=1, inplace=True)

C:\Users\deepa\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [132]:
toronto_neighborhood1.shape

(211, 3)

### Checking an instance where Postcode is same

In [133]:
#toronto_neighbourhood1.groupby('Postcode').count()
toronto_neighborhood1[toronto_neighborhood1["Postcode"]=="M1B"]

,Postcode,Borough,Neighborhood
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Checking how many rows have unassigned Neighbourhood

In [134]:
toronto_neighborhood1[toronto_neighborhood1["Neighborhood"]=="Not assigned"]

,Postcode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


### Assigning Borough name as Neighbourhood name where Neighbourhood name is unassigned

In [135]:
toronto_neighborhood1["Neighborhood"]=toronto_neighborhood1.apply(lambda x : x["Borough"] if x["Neighborhood"] == "Not assigned" else x["Neighborhood"] , axis=1 )
toronto_neighborhood1.head(7)


C:\Users\deepa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


### Grouping all the neighbourhood according to Postcode and Borough and making new neighbourhood dataframe. The new dataframe is sorted and neighbourhood is joined by comma. In the end index is reset.

In [136]:
toronto_neighborhood2=toronto_neighborhood1.groupby(["Postcode","Borough"],sort=True)["Neighborhood"].agg(",".join).reset_index()

### Checking new dataframe

In [137]:
toronto_neighborhood2.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Checking is there is na values for Neighbourhood and Borough

In [138]:
toronto_neighborhood2[toronto_neighborhood2["Neighborhood"].isna()]

,Postcode,Borough,Neighborhood


In [139]:
toronto_neighborhood2[toronto_neighborhood2["Borough"].isna()]

,Postcode,Borough,Neighborhood


### Checking shape that is rows and columns for final neighbourhood dataframe

In [140]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighborhood2['Borough'].unique()),
       toronto_neighborhood2.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Getting the latitude and longitude data

In [141]:
lat_long_data=pd.read_csv("C:/Users/deepa/Downloads/Geospatial_Coordinates.csv")

FileNotFoundError: File b'C:/Users/deepa/Downloads/Geospatial_Coordinates.csv' does not exist

In [142]:
lat_long_data.rename({"Postal Code":"Postcode"},axis=1,inplace=True)

In [143]:
lat_long_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [144]:
lat_long_data.shape

(103, 3)

In [145]:
#toronto_geo_data=pd.concat([toronto_neighbourhood2,lat_long_data],axis=1)

### Merging toronto_neighbourhood2 and lat_long_data on Postcode

In [146]:
toronto_geo_data=pd.merge(toronto_neighborhood2,lat_long_data,on="Postcode")

In [147]:
toronto_geo_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [148]:
type(toronto_geo_data["Borough"])

pandas.core.series.Series

### Checking how many data has name toronto in it group by

In [149]:
toronto_geo_data[toronto_geo_data["Borough"].str.contains("Toronto")  | toronto_geo_data["Borough"].str.contains("toronto")].groupby("Borough").count()

,Postcode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
West Toronto,6,6,6,6


### Getting dataframe of city toronto having borough name Toronto in it

In [150]:
torontocity_data=toronto_geo_data[toronto_geo_data["Borough"].str.contains("Toronto")  | toronto_geo_data["Borough"].str.contains("toronto")]

In [151]:
torontocity_data.reset_index(inplace=True)

In [152]:
torontocity_data.drop("index",axis=1,inplace=True)

C:\Users\deepa\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [153]:
torontocity_data.shape

(38, 5)

In [154]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_data['Latitude'], toronto_geo_data['Longitude'], toronto_geo_data['Borough'], toronto_geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [155]:
address = 'City of Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of City of Toronto are 43.7170226, -79.4197830350134.


In [156]:
# create map of Toronto using latitude and longitude values
map_torontocity = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontocity_data['Latitude'], torontocity_data['Longitude'], torontocity_data['Borough'], torontocity_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontocity)  
    
map_torontocity

In [157]:
torontocity_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [158]:
torontocity_data.loc[0,"Neighborhood"]

'The Beaches'

### Define foursquare credentials

In [159]:
CLIENT_ID = 'QAAXEG33DEMRTLWRYYW43Y5AD1NQNVBAHYLJOKH5X3BWAPUW' # Foursquare ID
CLIENT_SECRET = 'ZF0ULWL3DYUWTOHCP2BMARVZ2GUPAK535I0XPASB4EUO3O0B' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QAAXEG33DEMRTLWRYYW43Y5AD1NQNVBAHYLJOKH5X3BWAPUW
CLIENT_SECRET:ZF0ULWL3DYUWTOHCP2BMARVZ2GUPAK535I0XPASB4EUO3O0B


### Let's get the latitude and longitude of this neighbourhood

In [160]:
neighborhood_latitude = torontocity_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = torontocity_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = torontocity_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Let's get the list of top venues in this neighbourhood

In [161]:
radius=500
LIMIT=100

# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QAAXEG33DEMRTLWRYYW43Y5AD1NQNVBAHYLJOKH5X3BWAPUW&client_secret=ZF0ULWL3DYUWTOHCP2BMARVZ2GUPAK535I0XPASB4EUO3O0B&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

### Set the Get request and examine the result

In [162]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6b12a4bbed210032e2eaff'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

### Creating categories and dataframe for each category

In [163]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [164]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [165]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the neighborhoods in Toronto City

In [166]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [167]:
# type your answer here

toronto_venues = getNearbyVenues(names=torontocity_data['Neighborhood'],
                                   latitudes=torontocity_data['Latitude'],
                                   longitudes=torontocity_data['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [168]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [169]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [170]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


### Analyzing each neighbourhood

In [171]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's examine the shape of this dataframe

In [172]:
toronto_onehot.shape

(1706, 236)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [173]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.066667,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.000000,0.00,0.000000,0.000000,0.00,0.011628,0.000000,0.000000,0.011628,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01,0.000000,0.000000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.011628,0.00,0.000000,0.000000,0.00,0.000000,0.011628,0.011628,0.000000,0.011628


### Let's confirm the new size

In [174]:
toronto_grouped.shape

(38, 236)

#### Let's print each neighborhood along with the top 5 most common venues

In [175]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2              Garden  0.06
3       Auto Workshop  0.06
4      Farmers Market  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0   Airport Lounge  0.13
1  Airport Service  0.13
2      Coffee Shop  0.07
3            Plane  0.07
4              Bar  0.07


-

#### Let's put that into a *pandas* dataframe

Let's first write the function to sort the venues in descending order

In [176]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [177]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Cosmetics Shop,Asian Restaurant,Burger Joint,Hotel,Thai Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery,Belgian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Intersection,Convenience Store,Pet Store,Gym / Fitness Center,Nightclub,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Harbor / Marina,Airport Terminal,Sculpture Garden,Coffee Shop,Bar,Boat or Ferry,Plane,Airport Gate


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [178]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       2, 0, 4, 0, 0, 4, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [179]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontocity_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.tail() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,0,Coffee Shop,Café,Breakfast Spot,Gym,Intersection,Convenience Store,Pet Store,Gym / Fitness Center,Nightclub,Climbing Gym
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,0,Café,Mexican Restaurant,Thai Restaurant,Italian Restaurant,Music Venue,Discount Store,Bakery,Diner,Fried Chicken Joint,Flea Market
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,0,Breakfast Spot,Gift Shop,Italian Restaurant,Dessert Shop,Eastern European Restaurant,Bar,Bank,Dog Run,Movie Theater,Restaurant
36,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,0,Café,Coffee Shop,Pizza Place,Tea Room,Sushi Restaurant,Restaurant,Diner,Italian Restaurant,Food & Drink Shop,French Restaurant
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park


## Getting maps of Toronto city cluster wise

In [180]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters



Now, We can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. 

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Spa,Bookstore,Brewery,Bubble Tea Shop
2,East Toronto,0,Pizza Place,Pet Store,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Park,Sushi Restaurant
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
5,Central Toronto,0,Clothing Store,Breakfast Spot,Dog Run,Food & Drink Shop,Hotel,Park,Sandwich Place,Grocery Store,Gym,Electronics Store
6,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Diner,Miscellaneous Shop,Dessert Shop,Park,Chinese Restaurant
7,Central Toronto,0,Pizza Place,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Gym,Deli / Bodega,Greek Restaurant
9,Central Toronto,0,Pub,Coffee Shop,Pizza Place,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Restaurant,Liquor Store,Fried Chicken Joint
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pizza Place,Pub,Pharmacy,Bakery,Park,Italian Restaurant,Pet Store
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
13,Downtown Toronto,0,Coffee Shop,Café,Bakery,Park,Breakfast Spot,Mexican Restaurant,Theater,Gym / Fitness Center,Pub,Performing Arts Venue


In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].count()

Borough                   32
Cluster Labels            32
1st Most Common Venue     32
2nd Most Common Venue     32
3rd Most Common Venue     32
4th Most Common Venue     32
5th Most Common Venue     32
6th Most Common Venue     32
7th Most Common Venue     32
8th Most Common Venue     32
9th Most Common Venue     32
10th Most Common Venue    32
dtype: int64

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Wings Joint,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Bus Line,Park,Lawyer,Swim School,Wings Joint,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [185]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Health Food Store,Trail,Pub,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Playground,Park,Summer Camp,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,4,Park,Playground,Trail,Building,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
23,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## We can see that we have 32 observations in cluster 0, 1 in cluster 1, 1 in cluster 2, 1 in cluster 3 and 3 in cluster 4